In [17]:
import torch
import clip
import torch.nn as nn
import pickle
import torch.nn.functional as F
import os
import shutil
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
import cv2
from tqdm import tqdm


In [9]:
hopfield_memory = pickle.load(open('median_matrix.pkl','rb'))
#model,preprocess = clip.load()

In [33]:
context_img_folder_1 = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_images"
context_img_folder_2 = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_images_2"

context_label_folder_1 = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_proposal_label" 
context_label_folder_2 = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_proposal_label_2"

context_image_folder = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_images_final"
context_label_folder = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_proposal_label_final"

context_obb_folder = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_labels"
context_obb_folder_2 = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_labels_2"

context_obb_folder_final = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\best_context_labels_final"

os.makedirs(context_image_folder,exist_ok=True)
os.makedirs(context_label_folder,exist_ok=True)
os.makedirs(context_obb_folder_final,exist_ok=True)

In [ ]:
for images in os.listdir(context_img_folder_1):
    shutil.copy(os.path.join(context_img_folder_1,images),os.path.join(context_image_folder,"f1_"+images))
for images in os.listdir(context_img_folder_2):
    shutil.copy(os.path.join(context_img_folder_2,images),os.path.join(context_image_folder,"f2_"+images))    



In [4]:
for labels in os.listdir(context_label_folder_1):
    shutil.copy(os.path.join(context_label_folder_1,labels),os.path.join(context_label_folder,"f1_"+labels))
for labels in os.listdir(context_label_folder_2):
    shutil.copy(os.path.join(context_label_folder_2,labels),os.path.join(context_label_folder,"f2_"+labels))

In [3]:
for labels in os.listdir(context_obb_folder):
    shutil.copy(os.path.join(context_obb_folder,labels),os.path.join(context_obb_folder_final,"f1_"+labels))
for labels in os.listdir(context_obb_folder_2):
    shutil.copy(os.path.join(context_obb_folder_2,labels),os.path.join(context_obb_folder_final,"f2_"+labels))

In [9]:
# Where you want to save the new .pkl files
output_pkl_folder = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\merged_pickle_labels"
os.makedirs(output_pkl_folder, exist_ok=True)

# 1. Loop through each image in your image folder
for file in os.listdir(context_image_folder):
    if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
        
    image_base_name = file.rsplit('.', 1)[0]
    storage_dict = {}
    obj_indx = 0
    
    # 2. Build the filename for the first object (e.g., image_obj0.npz)
    obj_filename = f"{image_base_name}_obj{obj_indx}.npz"
    obj_path = os.path.join(context_label_folder, obj_filename)

    # 3. Collect all objects for this specific image
    while os.path.exists(obj_path):
        with np.load(obj_path) as data:
            # We store them exactly as they are (arrays and all)
            storage_dict[obj_indx] = {
                'class_id': data['cls'],
                'embedding': data['embedding']
            }
        
        # Increment to look for the next object (obj1, obj2...)
        obj_indx += 1
        obj_filename = f"{image_base_name}_obj{obj_indx}.npz"
        obj_path = os.path.join(context_label_folder, obj_filename)

    # 4. Save the merged dictionary as a Pickle file if objects were found
    if storage_dict:
        pkl_save_path = os.path.join(output_pkl_folder, f"{image_base_name}.pkl")
        
        with open(pkl_save_path, 'wb') as f:
            # HIGHEST_PROTOCOL makes it faster and smaller for large arrays
            pickle.dump(storage_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
            
        print(f"Merged {obj_indx} objects into → {image_base_name}.pkl")

print("All images processed and merged into individual Pickle files!")

Merged 2 objects into → f1_merged_100_horizontal.pkl
Merged 2 objects into → f1_merged_101_vertical.pkl
Merged 2 objects into → f1_merged_103_horizontal.pkl
Merged 2 objects into → f1_merged_104_vertical.pkl
Merged 3 objects into → f1_merged_107_horizontal.pkl
Merged 6 objects into → f1_merged_108_vertical.pkl
Merged 3 objects into → f1_merged_110_vertical.pkl
Merged 6 objects into → f1_merged_113_horizontal.pkl
Merged 4 objects into → f1_merged_118_horizontal.pkl
Merged 2 objects into → f1_merged_120_vertical.pkl
Merged 2 objects into → f1_merged_121_vertical.pkl
Merged 3 objects into → f1_merged_123_horizontal.pkl
Merged 2 objects into → f1_merged_128_horizontal.pkl
Merged 2 objects into → f1_merged_129_horizontal.pkl
Merged 2 objects into → f1_merged_12_horizontal.pkl
Merged 2 objects into → f1_merged_130_vertical.pkl
Merged 2 objects into → f1_merged_132_vertical.pkl
Merged 2 objects into → f1_merged_133_vertical.pkl
Merged 2 objects into → f1_merged_139_horizontal.pkl
Merged 2 obj

In [34]:
context_output_pkl_folder = r"C:\Users\surya\Desktop\computer vision\Hopfield networks\multi_context_pickles"

os.makedirs(context_output_pkl_folder, exist_ok=True)

# ---------------- CLIP SETUP ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
model.eval()

# ---------------- HELPER FUNCTIONS ----------------

def get_clip_embedding(cv2_img):
    """Converts CV2 BGR image to CLIP embedding with Zero-Size Protection."""
    if cv2_img is None or cv2_img.size == 0 or cv2_img.shape[0] == 0 or cv2_img.shape[1] == 0:
        return np.zeros(512, dtype=np.float32)

    rgb_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb_img)
    
    img_input = preprocess(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = model.encode_image(img_input)
        emb = emb / emb.norm(dim=-1, keepdim=True)
    return emb.squeeze(0).cpu().numpy()

def get_warped_object_crop(img_cv, src_pts):
    """Extracts a straightened (warped) crop of the object from OBB points."""
    width = int(np.linalg.norm(src_pts[0] - src_pts[1]))
    height = int(np.linalg.norm(src_pts[1] - src_pts[2]))
    width, height = max(1, width), max(1, height)

    dst_pts = np.array([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(img_cv, M, (width, height))
    return warped

def context_proposals(img_cv, obb_pts):
    """Generates the 4 context proposals as numpy arrays."""
    H, W = img_cv.shape[:2]
    xs, ys = [p[0] for p in obb_pts], [p[1] for p in obb_pts]
    
    xmin, xmax = max(0, int(min(xs))), min(W, int(max(xs)))
    ymin, ymax = max(0, int(min(ys))), min(H, int(max(ys)))
    
    bw, bh = xmax - xmin, ymax - ymin
    cx, cy = (xmin + xmax) // 2, (ymin + ymax) // 2
    
    exmin, exmax = max(0, cx - bw), min(W, cx + bw)
    eymin, eymax = max(0, cy - bh), min(H, cy + bh)

    return {
        '1': img_cv[:, xmin:xmax],             # Column
        '2': img_cv[ymin:ymax, :],             # Row
        '3': img_cv[eymin:eymax, exmin:exmax], # Box2x
        '4': img_cv                            # Full
    }

# ---------------- MAIN PROCESSING LOOP ----------------

images = [f for f in os.listdir(context_image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for image_file in images:
    image_path = os.path.join(context_image_folder, image_file)
    base_name = image_file.rsplit('.', 1)[0]
    print(image_file)
    obb_path = os.path.join(context_obb_folder_final, base_name + '.txt')

    if not os.path.exists(obb_path): 
        print(f"File not found:{base_name}")
        continue

    try:
        # Load image and convert to CV2 once
        pil_image = Image.open(image_path).convert("RGB")
        img_cv = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
        H, W = img_cv.shape[:2]
        
        with open(obb_path, 'r') as f:
            lines = f.read().splitlines()

        image_storage_dict = {}

        for i, line in enumerate(lines):
            parts = line.split()
            if len(parts) < 9: continue
            
            cls_id = int(parts[0])
            coords = [float(x) for x in parts[1:9]]
            
            # 1. Scale points
            src_pts = np.array([
                [coords[0]*W, coords[1]*H], [coords[2]*W, coords[3]*H],
                [coords[4]*W, coords[5]*H], [coords[6]*W, coords[7]*H]
            ], dtype="float32")

            # 2. Get Object Embedding (Straightened Crop)
            obj_crop = get_warped_object_crop(img_cv, src_pts)
            obj_emb = get_clip_embedding(obj_crop)

            # 3. Get Context Proposals
            proposals = context_proposals(img_cv, src_pts)

            # 4. Store everything
            image_storage_dict[i] = {
                'cls_id': cls_id,
                'object_embedding': obj_emb,
                '1': get_clip_embedding(proposals['1']),
                '2': get_clip_embedding(proposals['2']),
                '3': get_clip_embedding(proposals['3']),
                '4': get_clip_embedding(proposals['4'])
            }

        # 5. Save Pickle
        pkl_save_path = os.path.join(context_output_pkl_folder, f"{base_name}.pkl")
        with open(pkl_save_path, 'wb') as f:
            pickle.dump(image_storage_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

        print(f"Saved: {base_name}.pkl")

    except Exception as e:
        print(f"Error on {image_file}: {e}")

print("Processing Complete.")

f1_merged_100_horizontal.jpg
Saved: f1_merged_100_horizontal.pkl
f1_merged_101_vertical.jpg
Saved: f1_merged_101_vertical.pkl
f1_merged_103_horizontal.jpg
Saved: f1_merged_103_horizontal.pkl
f1_merged_104_vertical.jpg
Saved: f1_merged_104_vertical.pkl
f1_merged_107_horizontal.jpg
Saved: f1_merged_107_horizontal.pkl
f1_merged_108_vertical.jpg
Saved: f1_merged_108_vertical.pkl
f1_merged_110_vertical.jpg
Saved: f1_merged_110_vertical.pkl
f1_merged_113_horizontal.jpg
Saved: f1_merged_113_horizontal.pkl
f1_merged_118_horizontal.jpg
Saved: f1_merged_118_horizontal.pkl
f1_merged_120_vertical.jpg
Saved: f1_merged_120_vertical.pkl
f1_merged_121_vertical.jpg
Saved: f1_merged_121_vertical.pkl
f1_merged_123_horizontal.jpg
Saved: f1_merged_123_horizontal.pkl
f1_merged_128_horizontal.jpg
Saved: f1_merged_128_horizontal.pkl
f1_merged_129_horizontal.jpg
Saved: f1_merged_129_horizontal.pkl
f1_merged_12_horizontal.jpg
Saved: f1_merged_12_horizontal.pkl
f1_merged_130_vertical.jpg
Saved: f1_merged_130_ver

In [67]:
class attn(nn.Module):
    def __init__(self, dim_size=64): # dim_size will be 64
        super().__init__()
        self.key = nn.Linear(dim_size, dim_size)
        self.query = nn.Linear(dim_size, dim_size)
        self.value = nn.Linear(dim_size, dim_size)

    def forward(self, x, y):
        # x and y are now both (b, 4, 64)
        K = self.key(x) 
        Q = self.query(y)
        V = self.value(x) 

        # Standard Attention Calculation
        d_k = K.size(-1)
        # (b, 4, 64) @ (b, 64, 4) -> (b, 4, 4)
        attn_weights = torch.softmax(torch.matmul(Q, K.transpose(-2, -1)) / (d_k**0.5), dim=-1)

        # (b, 4, 4) @ (b, 4, 64) -> (b, 4, 64)
        return torch.matmul(attn_weights, V)

In [63]:
class MHA(nn.Module):
    def __init__(self, num_heads=8, dim_size=512):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = dim_size // num_heads # 512 / 8 = 64
        
        self.MHA = nn.ModuleList([attn(self.head_dim) for _ in range(num_heads)])
        self.norm = nn.LayerNorm(dim_size)
    
    def forward(self, x, y):
        # x shape: (b, 4, 512)
        # y shape: (b, 4, 512)
        
        attn_outs = []
        for i, head in enumerate(self.MHA):
            # Calculate the start and end indices for this head
            start = i * self.head_dim
            end = (i + 1) * self.head_dim
            
            # Slice the 512 dimensions into 64 dimensions for this specific head
            x_slice = x[:, :, start:end] # (b, 4, 64)
            y_slice = y[:, :, start:end] # (b, 4, 64)
            
            attn_outs.append(head(x_slice, y_slice))
            
        # Concatenate the 8 outputs of (b, 4, 64) back into (b, 4, 512)
        out = torch.cat(attn_outs, dim=-1)
        
        # Residual connection and normalization
        out = self.norm(out + y) 
        return out

In [68]:
class MLP(nn.Module):
    def __init__(self, dim_size=512, hidden_dim=1024):
        super().__init__()
        self.fc1 = nn.Linear(dim_size, hidden_dim)
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(hidden_dim, dim_size, bias=False)
        self.gelu = nn.GELU()
        self.norm = nn.LayerNorm(dim_size)

    def forward(self, x):
        residual = x
        out = self.fc1(x)
        out = self.gelu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.norm(out + residual) # Corrected: apply norm to result
        return out

In [69]:
class ContextAggregator(nn.Module):
    def __init__(self, dim_size=512):
        super().__init__()
        # Learnable weights to decide which of the 4 contexts is important
        self.query_projection = nn.Linear(dim_size, 1)
        self.softmax = nn.Softmax(dim=1)
        self.out_projection = nn.Sequential(
            nn.Linear(dim_size, dim_size),
            nn.LayerNorm(dim_size),
            nn.GELU()
        )

    def forward(self, x):
        """
        Input x: (b, 4, 512) - Output of your MHA modules
        Returns: (b, 1, 512) - The final combined context vector
        """
        # 1. Compute attention scores for each of the 4 proposals
        # (b, 4, 512) -> (b, 4, 1)
        attention_scores = self.query_projection(x)
        weights = self.softmax(attention_scores) # (b, 4, 1)

        # 2. Weighted sum of the 4 context vectors
        # (b, 1, 4) @ (b, 4, 512) -> (b, 1, 512)
        combined = torch.matmul(weights.transpose(-2, -1), x)

        # 3. Final refinement
        out = self.out_projection(combined)
        return out

In [70]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.MHAmoduoles = nn.ModuleList([MHA(num_heads=8, dim_size=512) for _ in range(4)])
        self.mlp = MLP()
        
        # ADDED: The aggregator to collapse (b, 4, 512) -> (b, 1, 512)
        self.aggregator = ContextAggregator(dim_size=512)

    def forward(self, x, y):
        # 1. Process through MHA layers (Interaction between object and contexts)
        for mha in self.MHAmoduoles:
            result = mha(x, y)
            x = x + result # Shape: (b, 4, 512)
        # 2. Refine with MLP
        x = self.mlp(x) # Shape: (b, 4, 512)

        # 3. Collapse to final vector
        # This combines the 4 views into 1 final prediction
        final_prediction = self.aggregator(x) # Shape: (b, 1, 512)
        
        return final_prediction

In [46]:
class CustomDataset(Dataset):
    def __init__(self, target_pkl_folder, training_pkl_folder):
        self.target_pkl_folder = target_pkl_folder
        self.training_pkl_folder = training_pkl_folder

        self.files_list = os.listdir(self.target_pkl_folder)

    def __len__(self):
        return len(self.files_list)
    
    def __getitem__(self, idx):
        # Use the pre-loaded list
        file_name = self.files_list[idx]

        target_pkl_path = os.path.join(self.target_pkl_folder,file_name)
        training_pkl_path = os.path.join(self.training_pkl_folder,file_name)

        # Load Pickle data (The merged context/embeddings)
        with open(target_pkl_path, 'rb') as f:
            target_context_data = pickle.load(f) #contains class_id target_embedding
        
        # Load OBB data (All lines/objects)
        with open( training_pkl_path, 'rb') as of:
            training_context_data = pickle.load(of)  #contains class_id object_embedding 1 2 3 4
       

        return target_context_data,training_context_data

In [71]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 50

# Ensure your model architecture (attn, MHA, MLP, model) is defined before this
model_net = model().to(device)
optimizer = torch.optim.AdamW(params=model_net.parameters(), lr=1e-4)

# Define loss function for (1, 1, 512) vs (1, 1, 512)
def loss_fn(pred, target):
    # Squeeze to (1, 512)
    p = pred.view(1, -1)
    t = target.view(1, -1)
    cos_sim = F.cosine_similarity(p, t, dim=-1)
    return (1 - cos_sim).mean()

# Initialize Dataset
train_dataset = CustomDataset(
    target_pkl_folder=output_pkl_folder,
    training_pkl_folder=context_output_pkl_folder
)

# 2. --- Training Loop ---
for epoch in range(epochs):
    model_net.train()
    total_loss = 0
    
    # Progress bar across all images in the dataset
    loop = tqdm(range(len(train_dataset)), desc=f"Epoch {epoch+1}/{epochs}")
    
    for idx in loop:
        # Direct access to raw dictionaries from Dataset __getitem__
        t_dict, tr_dict = train_dataset[idx]
        
        image_obj_loss = 0
        num_objs = len(t_dict.keys())

        # Iterate through each object index within the current image
        for obj_idx in t_dict.keys():
            
            # --- 1. Prepare Target ---
            target_emb = t_dict[obj_idx]['embedding']
            if isinstance(target_emb, np.ndarray):
                target_emb = torch.from_numpy(target_emb)
            target_emb = target_emb.float().to(device).view(1, 1, 512)

            # --- 2. Prepare Inputs ---
            train_obj = tr_dict[obj_idx]
            
            # Object Query (The Anchor)
            obj_query = train_obj['object_embedding']
            if isinstance(obj_query, np.ndarray):
                obj_query = torch.from_numpy(obj_query)
            obj_query = obj_query.float().to(device).view(1, 1, 512)

            # Context Proposals (The Candidates)
            c_list = []
            for key in ['1', '2', '3', '4']:
                c_vec = train_obj[key]
                if isinstance(c_vec, np.ndarray):
                    c_vec = torch.from_numpy(c_vec)
                c_list.append(c_vec.float())
            
            # Stack into (1, 4, 512)
            contexts = torch.stack(c_list, dim=0).unsqueeze(0).to(device)

            # --- 3. Forward & Backprop ---
            optimizer.zero_grad()
            
            # Forward pass through MHA -> MLP -> Aggregator
            prediction = model_net(contexts, obj_query) 
            
            loss = loss_fn(prediction, target_emb)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            image_obj_loss += loss.item()

        # Update progress bar with average loss for current image objects
        loop.set_postfix(avg_obj_loss=f"{(image_obj_loss/num_objs):.4f}")

    avg_epoch_loss = total_loss / len(train_dataset)
    print(f"-> Epoch {epoch+1} Completed. Avg Dataset Loss: {avg_epoch_loss:.4f}")

# 3. --- Save Results ---
torch.save(model_net.state_dict(), "hopfield_context_model.pth")
print("Model saved as hopfield_context_model.pth")

Epoch 1/50: 100%|██████████| 719/719 [02:03<00:00,  5.84it/s, avg_obj_loss=0.4227]


-> Epoch 1 Completed. Avg Dataset Loss: 1.7232


Epoch 2/50: 100%|██████████| 719/719 [01:47<00:00,  6.71it/s, avg_obj_loss=0.1551]


-> Epoch 2 Completed. Avg Dataset Loss: 0.6443


Epoch 3/50: 100%|██████████| 719/719 [01:53<00:00,  6.36it/s, avg_obj_loss=0.1335]


...

-> Epoch 48 Completed. Avg Dataset Loss: 0.0932


Epoch 49/50: 100%|██████████| 719/719 [01:45<00:00,  6.80it/s, avg_obj_loss=0.0447]


-> Epoch 49 Completed. Avg Dataset Loss: 0.0922


Epoch 50/50: 100%|██████████| 719/719 [01:47<00:00,  6.67it/s, avg_obj_loss=0.0445]

-> Epoch 50 Completed. Avg Dataset Loss: 0.0911
Model saved as hopfield_context_model.pth
